In [ ]:
%load_ext autoreload
%autoreload 2

from os import listdir
from os.path import isfile, join

import numpy as np
import matplotlib.pyplot as plt


import sys
sys.path.append('../src')
from preprocessing import  load_rhythm_feature_db

from models import OLSPatchRegressor
na = np.newaxis

plt.rc('text', usetex=True)
plt.rc('font', family='serif')


In [ ]:
X, y = load_rhythm_feature_db('../data/music_speech/music_wav', '../data/music_speech/speech_wav', 1, reload=True)
print(X.shape)
print(y.shape)
